See [http://www.nltk.org/book/ch08.html](http://www.nltk.org/book/ch08.html) for more information and [http://www.nltk.org/howto/parse.html](http://www.nltk.org/howto/parse.html) for extensive python examples.

Credit to Professor Culotta for most of the content

# Context-free Grammar

* Set of **rules** or **productions**
    * Define how constituents can be grouped
* **Lexicon**: list of words and symbols

## Example: CFG for Noun Phrases
```
NP → Det Nominal
NP → ProperNoun
Nominal → Noun | Noun Nominal
```

Rules can be part of a hierarchy:
```
Det → a
Det → the
Noun → flight
```

* **Terminal** symbols: words in the language (e.g., "a", "flight")
* **Nonterminal** symbols: clusters or generalizations of terminals (e.g., Noun, Nominal, NP)

## Derivation
* A sequence of rule expansions to generate a given string.
* This sequence is most commonly shown as a parse tree
![parse1](parse1.png)

Derivation

    1) NP → Det Nom
    2) Det → a
    3) Nom → Noun
    4) Noun → flight

### Example

![grammar](grammar.png)
![lexicon](lexicon1.png)
![parse2](parse2.png)

## Top-down Parsing

* Start at the root node S
* Expand rules until a word is reached at a leaf node
* If a rule fails to match a word, backtrack and try a different rule

### Example
![topdown1](topdown1.png)
![topdown2](topdown2.png)
![topdown3](topdown3.png)

In [5]:
import nltk

In [3]:
# Defining grammars in NLTK.
grammar = nltk.CFG.fromstring("""
  S -> NP VP | Aux NP VP | VP
  NP -> Det Nominal
  Nominal -> Noun | Noun Nominal
  NP -> ProperNoun
  VP -> Verb
  VP -> Verb NP
  Det -> 'that' | 'this' | 'a'
  Noun -> 'book' | 'flight' | 'meal' | 'money'
  Verb -> 'include'| 'book' | 'prefer'
  Aux -> 'does'
  Prep -> 'from' | 'to' | 'on'
  ProperNoun -> 'Houston' | 'TWA'
  """)

grammar

<Grammar with 25 productions>

In [8]:
# See the docstring for the nltk CFG object
grammar?

# See the source code for nltk CFG class
grammar??

In [10]:
# Create an nltk parser based off of the predefined grammar
parser = nltk.RecursiveDescentParser(grammar)
# Tokenize the sentence
sent = 'book that flight'.split()

# perform a depth-first search
[str(p) for p in parser.parse(sent)][0]

'(S (VP (Verb book) (NP (Det that) (Nominal (Noun flight)))))'

In [7]:
# To see the full parsing process, include trace=2
parser = nltk.RecursiveDescentParser(grammar, trace=2)
[str(p) for p in parser.parse(sent)]

Parsing 'book that flight'
    [ * S ]
  E [ * NP VP ]
  E [ * Det Nominal VP ]
  E [ * 'that' Nominal VP ]
  E [ * 'this' Nominal VP ]
  E [ * 'a' Nominal VP ]
  E [ * ProperNoun VP ]
  E [ * 'Houston' VP ]
  E [ * 'TWA' VP ]
  E [ * Aux NP VP ]
  E [ * 'does' NP VP ]
  E [ * VP ]
  E [ * Verb ]
  E [ * 'include' ]
  E [ * 'book' ]
  M [ 'book' ]
  E [ * 'prefer' ]
  E [ * Verb NP ]
  E [ * 'include' NP ]
  E [ * 'book' NP ]
  M [ 'book' * NP ]
  E [ 'book' * Det Nominal ]
  E [ 'book' * 'that' Nominal ]
  M [ 'book' 'that' * Nominal ]
  E [ 'book' 'that' * Noun ]
  E [ 'book' 'that' * 'book' ]
  E [ 'book' 'that' * 'flight' ]
  M [ 'book' 'that' 'flight' ]
  + [ 'book' 'that' 'flight' ]
  E [ 'book' 'that' * 'meal' ]
  E [ 'book' 'that' * 'money' ]
  E [ 'book' 'that' * Noun Nominal ]
  E [ 'book' 'that' * 'book' Nominal ]
  E [ 'book' 'that' * 'flight' Nominal ]
  M [ 'book' 'that' 'flight' * Nominal ]
  E [ 'book' 'that' 'flight' * Noun ]
  E [ 'book' 'that' 'flight' * 'book' ]
  E

['(S (VP (Verb book) (NP (Det that) (Nominal (Noun flight)))))']

### Three big issues:
* Left recursion leads to infinite recursion
* Ambiguity leads to many valid trees
* Repeated work
    * Many subtrees are repeated due to backtracking

## Bottom-up Parsing

* Start with input words and build trees up from there
* Valid if the parse ends at root symbol S

![bottom1](bottom1.png)

![bottom11](bottom11.png)

![bottom12](bottom12.png)

![bottom2](bottom2.png)

### Three big issues:
* Inefficient when grammar has a lot of ambiguity
* Repeated work
* Can often take exponential time in sentence length

## Dynamic Programming

* Solves problem by incrementally solving subproblems
* Reuses work from subproblems to solve larger problems
* Usually uses a table to store subproblem solutions


In [13]:
# ipythong "magic" commands
# https://ipython.org/ipython-doc/3/interactive/magics.html
%time
## Recursive solution
def fib(n):
    if n <= 1:  # base cases
      return n
    return fib(n-1) + fib(n-2)

fib(20)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 24.1 µs


6765

![fib](fib.jpg)

In [14]:
## dynamic program
## "table" just consists of two numbers, x and y
## See also here:
## http://www.geeksforgeeks.org/program-for-nth-fibonacci-number/
%time
def fib_dp(n):
    x = 0
    y = 1
    if n < 0:
        print("Incorrect input")
    elif n == 0:
        return x
    elif n == 1:
        return y
    else:
        for i in range(2, n+1):
            z = x + y
            x = y
            y = z
        return y
fib_dp(20)

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 6.91 µs


6765

## Earley Parser

* Top-down dynamic programming based parser
* Fills an array (called a **chart**) with N-1 entries in a single left-to-right pass
    * N is number of words in input
* For each word, chart contains list of states storing the partial parse

**Dot notation** is used to indicate progress in completing the parse. 

Each EP state contains:
    * A subtree for a single grammar rule
    * Information on the progress made in completing the subtree
    * Position of the tree w.r.t. input
    
### Example

For "Book that flight":
![ep-states](ep-states.png)

  1) The first 0 indicates that this production begins at index 0 (the start of the input). The second 0 indicates where the dot is.
  
  2) The 1 indicates this NP starts at position 1 and a _Det_ has already been parse and a _Nominal_ is expected next.
  
  3) A VP has been parsed that spans the entire input
  
See also:
![ep-states2](ep-states2.png)

## Earley Operators
### Predictor
* Create new states
* Applied to any state that has a **non-terminal** to the right of the dot and is **not** a part of speech.
* Results in one new state for each alternative expansion
* Added to the same chart entry as the generating state

Apply Predictor to S→*VP,[0,0]

Add new states VP→*_Verb_,[0,0] and

VP→*_Verb_ NP,[0,0] to first chart entry.


### Scanner
* Creates a new state that moves the dot past the POS tag
* Applied to states with a POS tag to the right of the dot
* Only valid POS tags will appear in the chart

Apply Scanner to VP→*_Verb_ NP,[0,0]

Add new state to second chart entry

VP→_Verb_*NP,[0,1]

### Completer
* Creates new state by copying older state, advancing dot, and adding to the current chart entry
* Applied to a state that has a dot that has reached the right end of a rule
* Represent a successful parse for a **constituent** for a span of the input
* Finds and advances all previously created states that were looking for this constituent at this position

Apply Completer to NP→Det Nominal*,[1,3]

Completer looks for states ending at 1 that expect an NP

Creates new state from VP→Verb*NP,[0,1] to

VP→Verb NP*,[0,3]

## Full Example
![topdown1](topdown1.png)
![ep-parse](ep-parse.png)

In [ ]:
from nltk.parse.earleychart import EarleyChartParser

In [32]:
nltk.parse.earleychart??

In [33]:
grammar = nltk.CFG.fromstring("""
  S -> NP VP | Aux NP VP | VP
  NP -> Det Nominal
  Nominal -> Noun | Noun Nominal
  NP -> ProperNoun
  VP -> Verb
  VP -> Verb NP
  Det -> 'that' | 'this' | 'a'
  Noun -> 'book' | 'flight' | 'meal' | 'money'
  Verb -> 'include'| 'book' | 'prefer'
  Aux -> 'does'
  Prep -> 'from' | 'to' | 'on'
  ProperNoun -> 'Houston' | 'TWA'
  """)
sentence = "book that flight"
tokens = sentence.split()
earley = EarleyChartParser(grammar, trace=10)
chart = earley.chart_parse(tokens)
parses = list(chart.parses(grammar.start()))

for tree in parses: print(tree)


|.    book   .    that   .   flight  .|
Leaf Init Rule:
|[-----------]           .           .| [0:1] 'book'
|.           [-----------]           .| [1:2] 'that'
|.           .           [-----------]| [2:3] 'flight'
Top Down Init Rule:
|>           .           .           .| [0:0] S  -> * NP VP
|>           .           .           .| [0:0] S  -> * Aux NP VP
|>           .           .           .| [0:0] S  -> * VP

* Processing queue: 0 

Predictor Rule:
|>           .           .           .| [0:0] VP -> * Verb
|>           .           .           .| [0:0] VP -> * Verb NP
Predictor Rule:
|>           .           .           .| [0:0] Verb -> * 'book'
Predictor Rule:
|>           .           .           .| [0:0] NP -> * Det Nominal
|>           .           .           .| [0:0] NP -> * ProperNoun

* Processing queue: 1 

Scanner Rule:
|[-----------]           .           .| [0:1] Verb -> 'book' *
Completer Rule:
|[-----------]           .           .| [0:1] VP -> Verb *
|[----------->   

In [25]:
grammar?